# Data Subsetting and Transformation Services in the Cloud

## Using the Harmony-Py library to access customized data from NASA Earthdata 

## Summary

Harmony allows you to seamlessly analyze Earth observation data from different NASA data centers... 
https://harmony.earthdata.nasa.gov/


#### Benefits

- Consistent access patterns to EOSDIS holdings make cross-data center data access easier
- Data reduction services allow users to request only the data they want, in the format and projection they want
- Analysis Ready Data and cloud access will help reduce time-to-science
- Community Development helps reduce the barriers for re-use of code and sharing of domain knowledge

Harmony-Py is a Python library for integrating with NASA's Harmony Services.

Harmony-Py provides a Python alternative to directly using Harmony's RESTful API. It handles NASA Earthdata Login (EDL) authentication and optionally integrates with the CMR Python Wrapper by accepting collection results as a request parameter. It's convenient for scientists who wish to use Harmony from Jupyter notebooks as well as machine-to-machine communication with larger Python applications.

### Objectives

1. Practice skills learned from intro to CMR tutorial to discover what access and service options exist for a given data set
2. 
___

## Import Packages

In [ ]:
#

## Discover service options for a given data set

- First, what do we mean by a "service"?
[TODO] Describe how we define services and their benefits, and how not all datasets have services on them due to level of support, etc....


- Let's see what the collection metadata tells us 

Building off of CMR introduction tutorial:

In [ ]:
#

We want to search by collection to inspect the access and service options that exist:

In [ ]:
#

In the CMR introduction tutorial, we explored cloud-hosted collections from different DAAC providers, and identified the CMR `concept-id` for a given data set id (also referred to as a `short_name`). 

Here we are jumping ahead and already know the `concept_id` we are interested in, by browsing cloud-hosted datasets from PO.DAAC in Earthdata Search: https://search.earthdata.nasa.gov/portal/podaac-cloud/search. 

We are going to focus on `MODIS_A-JPL-L2P-v2019.0`: GHRSST Level 2P Global Sea Surface Skin Temperature from the Moderate Resolution Imaging Spectroradiometer (MODIS) on the NASA Aqua satellite (GDS2). Let's first save this as a variable that we can use later on once we request data from Harmony.

In [ ]:
#

We will view the top-level metadata for this collection to see what additional service and variable metadata exist.

In [ ]:
#

Print the response:

In [ ]:
#

Let's walk through what each of these service values mean:

- Associations
     - CMR is a large web of interconnected metadata "schemas", including Collections, Granules, Services, Tools, and Variables. In this case, this collection is associated with two unique services, two tools, and several unique variables.
- Tags
    - There are also tags that describe what service options exist at a high-level. In this case, we see that this dataset supports the ability to reformat, subset by space and time, as well as by variable. This is used in web applications like Earthdata Search to surface those customization options more readily. 
- Service Features
    - In this case, we see three separate "features" listed here: esi, Harmony, and OPeNDAP.
    
We will dig into more details on what Harmony offers for this dataset.

First, we need to isolate the services returned for this dataset:

In [ ]:
#

In [ ]:
#

Inspect the first service returned. Now we're going to search the services endpoint to view that individual service's metadata, like we did with our dataset above.

TODO: Explain why we need the output format in umm_json

In [ ]:
#

In [ ]:
#

In [ ]:
#

TODO: Describe these different service options and broader Harmony / backend subsetter context.

## Discover variable names

TODO: Could this be an "exercise" to gain more familiarity with CMR? 

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

Let's print out a simple list of all associated variable names.

In [ ]:
#

In [ ]:
#

## Harmony-Py set up

[TODO] Describe Harmony-Py in more detail (connection between the library and the API).

Next steps adopted from the intro tutorial notebook in the Harmony-Py library: https://github.com/nasa/harmony-py/blob/main/examples/intro_tutorial.ipynb

### Create Harmony Client object
First, we need to create a Harmony Client, which is what we will interact with to submit and inspect a data request to Harmony, as well as to retrieve results.

When creating the Client, we need to provide Earthdata Login credentials, which are required to access data from NASA EOSDIS. This basic line below assumes that we have a `.netrc` available. 

In [ ]:
#

### Create Harmony Request
The following are common request parameters:

- `collection`: Required parameter. This is the NASA EOSDIS collection, or data product. There are two options for inputting a collection of interest:
    - Provide a concept ID, which is an ID provided in the Common Metadata Repository (CMR) metadata
    - Data product short name (e.g. SENTINEL-1_INTERFEROGRAMS).
- `spatial`: Bounding box spatial constraints on the data. The Harmony Bbox class accepts spatial coordinates as decimal degrees in w, s, e, n order, where longitude = -180, 180 and latitude = -90, 90.
- `temporal`: Date/time constraints on the data. The example below demonstrates temporal start and end ranges using the python datetime library.

Other advanced parameters that may be of interest. Note that many reformatting or advanced projection options may not be available for your requested dataset. See the [documentation](https://harmony-py.readthedocs.io/en/latest/) for details on how to construct these parameters.

[TODO] Describe area/time use case with this dataset - maybe demonstrating Earthdata Search for browsing??

In [ ]:
#

### Check Request validity
Before submitting a Harmony Request, we can test your request to see if it's valid and how to fix it if not. In particular, `request.is_valid` will check to ensure that the spatial BBox bounds and temporal ranges are entered correctly.

In [ ]:
#

### Submit request

Now that the request is created, we can now submit it to Harmony using the Harmony Client object. A job id is returned, which is a unique identifier that represents the submitted request.

In [ ]:
#

### Check request status

We can check on the progress of a processing job with status(). This method blocks while communicating with the server but returns quickly.

In [ ]:
#

Depending on the size of the request, it may be helpful to wait until the request has completed processing before the remainder of the code is executed. The wait_for_processing() method will block subsequent lines of code while optionally showing a progress bar.

In [ ]:
#

### View Harmony job response and output URLs
Once the data request has finished processing, we can view details on the job that was submitted to Harmony, including the API call to Harmony, and informational messages on the request if available.

result_json() calls wait_for_processing() and returns the complete job in JSON format once processing is complete. 

In [ ]:
#

### Direct cloud access

Note that the remainder of this tutorial will only succeed when running this notebook within the AWS us-west-2 region.

Harmony data outputs can be accessed within the cloud using the s3 URLs and AWS credentials provided in the Harmony job response.

#### Retrieve list of output URLs.

The result_urls() method calls wait_for_processing() and returns a list of the processed data URLs once processing is complete. You may optionally show the progress bar as shown below.

In [ ]:
#

We can see that the first file returned does not include the `_subsetted` suffix, which indicates that a blank file was returned, as no data values were located within our subsetted region. We'll select the second URL in the list to bring into xarray below.

In [ ]:
#

#### AWS credential retrieval
Using aws_credentials you can retrieve the credentials needed to access the Harmony s3 staging bucket and its contents.

In [ ]:
#

### Open staged files with *s3fs* and *xarray*

We use the AWS `s3fs` package to create a file system that can then be read by xarray:

In [ ]:
#

### **Note, Ideally, we'd show how to bring in all files into a single xarray ds but I'm getting an error:

In [ ]:
#

Now that we have our s3 file system set, including our declared credentials, we'll use that to open the url, and read in the file through xarray:

In [ ]:
#

### Plot the data

In [ ]:
#

### Extra code attempting to read in directly from s3 into xarray:

In [ ]:
#